
# Ce code prépare un environnement Python complet pour effectuer des tâches avancées de traitement de texte et d'intelligence artificielle. Il commence par installer une série de bibliothèques essentielles, notamment PyTorch pour le deep learning, Transformers pour les modèles de langage pré-entraînés, LangChain pour la construction d'applications conversationnelles, ChromaDB pour le stockage et la recherche efficace de données, PyPDF pour manipuler les fichiers PDF, et diverses autres bibliothèques spécialisées.

# Une fois les installations terminées, le code importe les principales bibliothèques nécessaires, couvrant des domaines tels que la génération de texte, l'extraction d'informations à partir de documents PDF, la création d'embeddings de texte et de phrases, et l'accélération des transformers. Il définit également le périphérique à utiliser (CPU ou GPU) en fonction de la disponibilité du GPU.

# Dans l'ensemble, ce code prépare un environnement puissant et flexible pour travailler sur des tâches complexes de traitement de texte et d'intelligence artificielle, en tirant parti des dernières avancées dans ces domaines. Il permet d'accéder à une vaste gamme d'outils et de techniques, offrant ainsi une base solide pour développer des applications innovantes.**texte en gras**

In [ ]:
!nvidia-smi

Fri Feb  9 15:37:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
gradio 4.17.0 requires pydantic>=2.0, but you have pydantic 1.10

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [ ]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [ ]:
!rm -rf "db"

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

1

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

1

In [ ]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 1.74 s, sys: 251 ms, total: 1.99 s
Wall time: 3.58 s


In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# La fonction process_pdf_request prend deux paramètres : pdf_path (le chemin vers le répertoire contenant les fichiers PDF) et user_request (la requête de l'utilisateur).

# Chargement des documents PDF :

# La bibliothèque PyPDFDirectoryLoader est utilisée pour charger tous les fichiers PDF du répertoire pdf_path.
# Les documents chargés sont stockés dans la variable docs.
# Découpage des documents en morceaux :

# La bibliothèque RecursiveCharacterTextSplitter est utilisée pour découper les documents en morceaux de 1024 caractères avec un chevauchement de 64 caractères. Les morceaux de texte sont stockés dans la variable texts.
# Création de la base de données vectorielle :

# La bibliothèque Chroma est utilisée pour créer une base de données vectorielle à partir des morceaux de texte.
# Les embeddings des morceaux de texte sont calculés et stockés dans la base de données, qui est persistée dans le répertoire "db".
# Préparation du modèle de génération de texte :

# Un objet TextStreamer est créé pour contrôler le flux de sortie du modèle de génération de texte.
# Un pipeline de génération de texte est créé avec le modèle et le tokenizer spécifiés, et avec des paramètres de génération tels que la température, la taille de la sortie, etc.
# Un objet HuggingFacePipeline est créé à partir du pipeline de génération de texte.
# Préparation de la chaîne de question-réponse :

# Un PromptTemplate est créé avec un modèle de prompt qui utilise les variables "context" et "question".
# Une chaîne de question-réponse RetrievalQA est créée en utilisant le modèle de génération de texte, la base de données vectorielle et le prompt.
# Exécution de la requête de l'utilisateur :

# La fonction qa_chain est appelée avec la requête de l'utilisateur, et le résultat est stocké dans la variable result.
# Retour du résultat :

# La fonction process_pdf_request retourne le résultat de la chaîne de question-réponse.
# Enfin, le code montre comment utiliser la fonction process_pdf_request en demandant à l'utilisateur d'entrer une requête, en appelant la fonction avec le chemin du répertoire PDF et la requête de l'utilisateur, et en affichant le résultat.

In [ ]:
def process_pdf_request(pdf_path, user_request):

    loader = PyPDFDirectoryLoader("pdfs")
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
    texts = text_splitter.split_documents(docs)

    db = Chroma.from_documents(texts, embeddings, persist_directory="db")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    text_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        temperature=0,
        top_p=0.95,
        repetition_penalty=1.15,
        streamer=streamer)

    llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt})

    result = qa_chain(user_request)

    return result

# Utilisation de la fonction
pdf_path = "pdfs"
user_request = input("Veuillez insérer votre demande : ")
result = process_pdf_request(pdf_path, user_request)
print(result)



Veuillez insérer votre demande : faite une extraction des information de cette facture en json?


The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

 Based on the provided invoice, here is a JSON representation of the information:

{
"date": "December 1, 2025",
"customer": {
"name": "Natalie Dufour",
"email": "bonjour@sitevraimentsuper.fr",
"address": {
"street": "12 rue de la Paix",
"city": "Paris",
"postcode": "75000"
},
"phone": "01 23 45 67 89"
},
"services": [
{
"description": "Service 1",
"quantity": "1",
"unitPrice": "123",
"total": "123"
},
{
"description": "Service 2",
"quantity": "1",
"unitPrice": "246",
"total": "246"
},
{
"description": "Service 3",
"quantity": "1",
"unitPrice": "123",
"total": "123"
}
],
"subTotal": "492",
"VAT": "0",
"total": "492"
}

Note that this JSON representation only includes the information that is present in the invoice, and does not include any additional fields or properties that might be included in a more comprehensive JSON representation of the invoice data.
{'query': 'faite une extraction des information de cette facture en json?', 'result': '  Based on the provided invoice, here is a J